In [1]:
import numpy as np
import pandas as pd
from glob import glob
import json
from decimal import Decimal
#%matplotlib inline
#import matplotlib.pyplot as plt
#import tensorflow as tf
from os.path import join
#import seaborn as sns
from collections import Counter

from sklearn.decomposition import TruncatedSVD


In [2]:
BASE_DIR= ''

In [3]:
json_path= join(BASE_DIR, 'nutrition_supplements_type')

In [190]:
def preprocess_nutrition_supplements(data_path: str, nutrition_type: list) -> list:
    nutrition_supplements = list()
    for category in nutrition_type:
        
        file_path = glob(join(data_path, category, '*'))
        #print(file_path)
        for file in file_path:
            with open(file, 'r', encoding='utf-8-sig') as json_file:
                file = json.load(json_file)
                file.pop('price')
                file.pop('serving')
                file.pop('url')
                file.pop('title')
                #file['type'] = category

                for nutr in file['nutrients']:
                    #print(nutr)
                    temp = file['nutrients'].get(nutr)[0]
                    #print(temp)
                    file[nutr] = temp
                nutrition_supplements.append(file)
                file.pop('nutrients')

    return nutrition_supplements

In [117]:
amino= preprocess_nutrition_supplements(json_path, ['amino-acids'])
mineral= preprocess_nutrition_supplements(json_path, ['minerals'])
vitamins= preprocess_nutrition_supplements(json_path, ['vitamins'])

In [108]:
a

[{'prod_cd': 'EVL-90914',
  'price': '27574',
  'Leucine': '2500',
  'Valine': '1250',
  'Iso_Leucine': '1250'},
 {'prod_cd': 'JYM-02141',
  'price': '46224',
  'Sodium': '80',
  'Carbohydrate': '1',
  'Calcium': '41',
  'Leucine': '3600.0',
  'Iso_Leucine': '1200.0',
  'Valine': '1200.0'},
 {'prod_cd': 'NOW-00265', 'price': '16862', 'Tyrosine': '0.35'},
 {'prod_cd': 'EVL-02003',
  'price': '31713',
  'Vitamin_C': '90',
  'Vitamin_B6': '10',
  'Vitamin_B12': '30',
  'Sodium': '39',
  'Leucine': '2500',
  'Iso_Leucine': '1250',
  'Valine': '1250'},
 {'prod_cd': 'LEX-12539',
  'price': '15050',
  'Leucine': '1200',
  'Iso_Leucine': '600',
  'Valine': '600'},
 {'prod_cd': 'MRM-71027',
  'price': '60825',
  'Vitamin_B6': '2',
  'Leucine': '2500',
  'Valine': '1500',
  'Iso_Leucine': '1000'},
 {'prod_cd': 'DRB-00110', 'price': '17375', 'Phenylalanine': '0.5'},
 {'prod_cd': 'BLB-01590',
  'price': '34020',
  'Protein': '3',
  'Leucine': '2000',
  'Iso_Leucine': '500',
  'Valine': '500'},
 {'

In [175]:
total= [
    'Protein', 'Fat', 'Carbohydrate', 'Dietary_Fiber', 'Calcium',
    'Iron', 'Magnesium', 'Phosphorus', 'Potassium', 'Sodium', 'Zinc',
    'Copper', 'Manganese', 'Selenium', 'Vitamin_A', 'Vitamin_D', 'Niacin',
    'Folic_acid', 'Vitamin_B12', 'Vitamin_B6', 'Vitamin_C', 'Vitamin_E',
    'Vitamin_K', 'Leucine', 'Iso_Leucine', 'Histidine', 'Linoleic_Acid', 
    'Alpha_Linolenic_Acid', 'Lysine', 'Methionine', 'Phenylalanine', 'Tyrosine',
    'Threonine', 'Valine', 'Cholesterol', 'Calories'
]

In [181]:
dfa= pd.DataFrame(amino, columns= ['prod_cd']+total+['price'], dtype= float)
dfa.set_index('prod_cd', inplace= True)

dfm= pd.DataFrame(mineral, columns= ['prod_cd']+total+['price'], dtype= float)
dfm.set_index('prod_cd', inplace= True)

dfv= pd.DataFrame(vitamins, columns= ['prod_cd']+total+['price'], dtype= float)
dfv.set_index('prod_cd', inplace= True)

/var/folders/pr/phpb7l1s675c1wqrhj9jc4lh0000gn/T/ipykernel_20943/2011201477.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  dfa= pd.DataFrame(amino, columns= ['prod_cd']+total+['price'], dtype= float)
/var/folders/pr/phpb7l1s675c1wqrhj9jc4lh0000gn/T/ipykernel_20943/2011201477.py:4: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  dfm= pd.DataFrame(mineral, columns= ['prod_cd']+total+['price'], dtype= float)
/var/folders/pr/phpb7l1s675c1wqrhj9jc4lh0000gn/T/ipykernel_20943/2011201477.py:7: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype

In [182]:
dfa.fillna(0, inplace= True)
dfv.fillna(0, inplace= True)
dfm.fillna(0, inplace= True)

In [183]:
dfa['Phenylalanine+Tyrosine']= dfa['Phenylalanine']+dfa['Tyrosine']
dfm['Phenylalanine+Tyrosine']= dfm['Phenylalanine']+dfm['Tyrosine']
dfv['Phenylalanine+Tyrosine']= dfv['Phenylalanine']+dfv['Tyrosine']

dfa.drop(['Phenylalanine', 'Tyrosine'], axis=1, inplace= True)
dfm.drop(['Phenylalanine', 'Tyrosine'], axis=1, inplace= True)
dfv.drop(['Phenylalanine', 'Tyrosine'], axis=1, inplace= True)

In [184]:
dfa.sort_values(by= 'price', ascending=False)[:10]

,Protein,Fat,Carbohydrate,Dietary_Fiber,Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,...,Linoleic_Acid,Alpha_Linolenic_Acid,Lysine,Methionine,Threonine,Valine,Cholesterol,Calories,price,Phenylalanine+Tyrosine
prod_cd,,,,,,,,,,,,,,,,,,,,,
OLY-00399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,730.0,0.0,0.0,0.0,0.0,0.0,66561.0,0.00
MRM-71027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1500.0,0.0,0.0,60825.0,0.00
OPN-02522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1250.0,0.0,0.0,55783.0,0.00
EVL-86451,0.0,0.0,0.0,0.0,180.0,0.0,0.0,0.0,150.0,170.0,...,0.0,0.0,0.0,0.0,0.0,1250.0,0.0,0.0,55170.0,0.00
THR-00464,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,650.0,50.0,350.0,625.0,0.0,0.0,52320.0,0.13
THR-00366,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,650.0,50.0,350.0,625.0,0.0,0.0,52320.0,0.13
BLB-00014,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,240.0,80.0,160.0,170.0,0.0,0.0,50130.0,0.21
DYZ-18102,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,...,0.0,0.0,2800.0,0.0,0.0,0.0,0.0,0.0,49380.0,0.00
DYZ-18101,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,...,0.0,0.0,2800.0,0.0,0.0,0.0,0.0,0.0,49380.0,0.00


In [185]:
dfm.sort_values(by= 'price', ascending=False)[:10]

,Protein,Fat,Carbohydrate,Dietary_Fiber,Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,...,Linoleic_Acid,Alpha_Linolenic_Acid,Lysine,Methionine,Threonine,Valine,Cholesterol,Calories,price,Phenylalanine+Tyrosine
prod_cd,,,,,,,,,,,,,,,,,,,,,
MCL-03069,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,121781.0,0.0
MCL-01995,0.0,0.0,0.0,0.0,180.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118970.0,0.0
LEM-01280,0.0,0.0,0.0,0.0,122.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92339.0,0.0
DIO-91325,0.0,1.3,0.0,0.0,160.0,0.0,80.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85160.0,0.0
MCL-03256,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70693.0,0.0
SNS-02486,0.0,0.0,0.0,0.0,0.0,0.0,147.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59966.0,0.0
THR-28002,0.0,0.0,0.0,0.0,500.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59621.0,0.0
GOL-11761,0.0,0.0,0.0,0.0,3300.0,0.0,60.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57905.0,0.0
MBM-00426,0.0,0.0,0.0,0.0,0.0,0.0,125.0,0.0,99.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55970.0,0.0


In [125]:
dfv.sort_values(by= 'price', ascending=False)[:10]

,Protein,Fat,Carbohydrate,Dietary_Fiber,Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,...,Linoleic_AcidAlpha_Linolenic_Acid,Lysine,Methionine,Phenylalanine,Tyrosine,Threonine,Valine,Cholesterol,Calories,price
prod_cd,,,,,,,,,,,,,,,,,,,,,
MCL-01996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111964.0
NBC-00203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108577.0
ZAH-08183,0.0,0.5,0.0,0.0,100.0,27.0,40.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106733.0
NCR-00318,0.0,0.0,0.0,0.0,75.0,27.0,15.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102414.0
MCL-01532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97974.0
NCR-90176,0.0,0.0,0.0,0.0,40.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97883.0
NCR-90148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97883.0
CAR-17520,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97340.0
AGE-12114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97328.0


In [286]:
df.shape

(2173, 33)

In [126]:
test_user_intake = {
    'Vitamin_B12': Decimal('3.8'),
    'Calcium': Decimal('277.2'),
    'Iron': Decimal('9.4'),
    'Leucine': Decimal('4352.6'),
    'Histidine': Decimal('1599.2'),
    'Phenylalanine+Tyrosine': Decimal('3694.2'),
    'Carbohydrate': Decimal('169.6'),
    'Magnesium': Decimal('162'),
    'Selenium': Decimal('72.6'),
    'Copper': Decimal('1'),
    'Threonine': Decimal('2486'),
    'Alpha_Linolenic_Acid': Decimal('1'),
    'Methionine': Decimal('830.6'),
    'Linoleic_Acid': Decimal('11.4'),
    'Phosphorus': Decimal('711.8'),
    'Calories': Decimal('1284'),
    'Protein': Decimal('60'),
    'Zinc': Decimal('18.8'),
    'Cholesterol': Decimal('380.4'),
    'Sodium': Decimal('2086.4'),
    'Vitamin_C': Decimal('18.2'),
    'Lysine': Decimal('3878.2'),
    'Valine': Decimal('2809.2'),
    'Vitamin_A': Decimal('7018.8'),
    'Fat': Decimal('38'),
    'Folic_acid': Decimal('365.4'),
    'Dietary_Fiber': Decimal('18'),
    'Potassium': Decimal('1618.2'),
    'Iso_Leucine': Decimal('2320.4'),
    'Manganese': Decimal('3.2')
    }

In [127]:
test_user_RDI = {
   'Vitamin_B12': Decimal('2.4'),
   'Calcium': Decimal('800'),
   'Iron': Decimal('10'),
   'Leucine': Decimal('3100'),
   'Histidine': Decimal('1000'),
   'Phenylalanine+Tyrosine': Decimal('3600'),
   'Carbohydrate': Decimal('248'),
   'Magnesium': Decimal('360'),
   'Selenium': Decimal('60'),
   'Copper': Decimal('850'),
   'Alpha_Linolenic_Acid': Decimal('1.6'),
   'Threonine': Decimal('1500'),
   'Linoleic_Acid': Decimal('13'),
   'Methionine': Decimal('1400'),
   'Phosphorus': Decimal('700'),
   'Niacin': Decimal('16'),
   'Zinc': Decimal('10'),
   'Calories': Decimal('1655'),
   'Protein': Decimal('70'),
   'Sodium': Decimal('1500'),
   'Vitamin_D': Decimal('10'),
   'Vitamin_C': Decimal('100'),
   'Lysine': Decimal('3100'),
   'Valine': Decimal('1700'),
   'Vitamin_A': Decimal('800'),
   'Fat': Decimal('42'),
   'Folic_acid': Decimal('400'),
   'Dietary_Fiber': Decimal('30'),
   'Vitamin_E': Decimal('12'),
   'Vitamin_B6': Decimal('1.5'),
   'Potassium': Decimal('3500'),
   'Vitamin_K': Decimal('75'),
   'Iso_Leucine': Decimal('1400'),
   'Manganese': Decimal('4')
    }

In [128]:
key= []
val= []
for k, v in test_user_RDI.items():
    key.append(k)
    val.append(v)
#val= np.array(val).reshape(-1,1)
df_user_RDI= pd.DataFrame(np.array(val).reshape(1,-1).tolist(), columns= key, dtype= float)

In [129]:
#df_user.append(np.array(val).reshape(1,-1).tolist())
#np.array(val)
df_user_RDI

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Carbohydrate,Magnesium,Selenium,Copper,...,Vitamin_A,Fat,Folic_acid,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
0,2.4,800.0,10.0,3100.0,1000.0,3600.0,248.0,360.0,60.0,850.0,...,800.0,42.0,400.0,30.0,12.0,1.5,3500.0,75.0,1400.0,4.0


### 영양제와 비교하기 위해 필요없는 컬럼 삭제

In [130]:
df_user_RDI.drop(['Calories', 'Folic_acid', 'Carbohydrate', 'Protein', 'Fat'], axis= 1,  inplace=True)

In [131]:
df_user_RDI

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
0,2.4,800.0,10.0,3100.0,1000.0,3600.0,360.0,60.0,850.0,1.6,...,3100.0,1700.0,800.0,30.0,12.0,1.5,3500.0,75.0,1400.0,4.0


### 기준 컬럼

In [132]:
standard_columns= df_user_RDI.columns

In [133]:
len(standard_columns)

29

In [134]:
sr_user_intake= pd.Series(test_user_intake, index= standard_columns, dtype= float)
df_user_intake= pd.DataFrame(sr_user_intake)

In [135]:
df_user_intake= df_user_intake.T

In [136]:
df_user_intake

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
0,3.8,277.2,9.4,4352.6,1599.2,3694.2,162.0,72.6,1.0,1.0,...,3878.2,2809.2,7018.8,18.0,NaN,NaN,1618.2,NaN,2320.4,3.2


### Difference= 권장량(RDI)- 섭취량(intake)

In [137]:
df_diff= df_user_RDI- df_user_intake
df_diff

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
0,-1.4,522.8,0.6,-1252.6,-599.2,-94.2,198.0,-12.6,849.0,0.6,...,-778.2,-1109.2,-6218.8,12.0,NaN,NaN,1881.8,NaN,-920.4,0.8


In [138]:
df_diff.fillna(0, inplace= True)

### 비율= (diff/ RDI)/ 100.

In [139]:
df_diff_ratio= (df_diff/ df_user_RDI)/ 100.
df_diff_ratio

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
0,-0.005833,0.006535,0.0006,-0.004041,-0.005992,-0.000262,0.0055,-0.0021,0.009988,0.00375,...,-0.00251,-0.006525,-0.077735,0.004,0.0,0.0,0.005377,0.0,-0.006574,0.002


### 영양제 데이터 컬럼 수정

In [301]:
df= pd.DataFrame(df, columns= standard_columns, dtype= float)
df

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
prod_cd,,,,,,,,,,,,,,,,,,,,,
NAP-03772,500.0,25.0,85.0,0.0,0.0,NaN,0.0,10.0,2.0,0.0,...,0.0,0.0,0.0,0.0,20.0,25.0,0.0,0.0,0.0,5.0
THR-22102,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NOW-01462,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0
SDN-41283,0.0,0.0,65.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NWY-41010,0.0,500.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CEN-21245,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,180.0,0.0,0.0,0.0,0.0,0.0
PAR-77883,100.0,10.0,0.0,0.0,0.0,NaN,10.0,70.0,0.5,0.0,...,0.0,0.0,1650.0,0.0,20.1,10.0,10.0,80.0,0.0,2.0
KIM-02098,5000.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [302]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2173 entries, NAP-03772 to SDN-90526
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Vitamin_B12             2173 non-null   float64
 1   Calcium                 2173 non-null   float64
 2   Iron                    2173 non-null   float64
 3   Leucine                 2173 non-null   float64
 4   Histidine               2173 non-null   float64
 5   Phenylalanine+Tyrosine  0 non-null      float64
 6   Magnesium               2173 non-null   float64
 7   Selenium                2173 non-null   float64
 8   Copper                  2173 non-null   float64
 9   Alpha_Linolenic_Acid    2173 non-null   float64
 10  Threonine               2173 non-null   float64
 11  Linoleic_Acid           0 non-null      float64
 12  Methionine              2173 non-null   float64
 13  Phosphorus              2173 non-null   float64
 14  Niacin                  2173 non

In [303]:
df.fillna(0, inplace= True)

In [304]:
df

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
prod_cd,,,,,,,,,,,,,,,,,,,,,
NAP-03772,500.0,25.0,85.0,0.0,0.0,0.0,0.0,10.0,2.0,0.0,...,0.0,0.0,0.0,0.0,20.0,25.0,0.0,0.0,0.0,5.0
THR-22102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NOW-01462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0
SDN-41283,0.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NWY-41010,0.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CEN-21245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,180.0,0.0,0.0,0.0,0.0,0.0
PAR-77883,100.0,10.0,0.0,0.0,0.0,0.0,10.0,70.0,0.5,0.0,...,0.0,0.0,1650.0,0.0,20.1,10.0,10.0,80.0,0.0,2.0
KIM-02098,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 비율= (영양제/ 섭취량(RDI))/ 100.

In [140]:
sr_user_RDI= pd.Series(test_user_RDI, index= standard_columns, dtype= float)

In [306]:
# 인덱스 명이 일치하지 않을 때는 시리즈 형태를 사용)
df_ratio= (df/ sr_user_RDI)/ 100.

In [307]:
df_ratio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2173 entries, NAP-03772 to SDN-90526
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Vitamin_B12             2173 non-null   float64
 1   Calcium                 2173 non-null   float64
 2   Iron                    2173 non-null   float64
 3   Leucine                 2173 non-null   float64
 4   Histidine               2173 non-null   float64
 5   Phenylalanine+Tyrosine  2173 non-null   float64
 6   Magnesium               2173 non-null   float64
 7   Selenium                2173 non-null   float64
 8   Copper                  2173 non-null   float64
 9   Alpha_Linolenic_Acid    2173 non-null   float64
 10  Threonine               2173 non-null   float64
 11  Linoleic_Acid           2173 non-null   float64
 12  Methionine              2173 non-null   float64
 13  Phosphorus              2173 non-null   float64
 14  Niacin                  2173 non

In [308]:
df_ratio

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
prod_cd,,,,,,,,,,,,,,,,,,,,,
NAP-03772,2.083333,0.000313,0.085,0.0,0.0,0.0,0.000000,0.001667,0.000024,0.0,...,0.0,0.0,0.000000,0.0,0.016667,0.166667,0.000000,0.000000,0.0,0.01250
THR-22102,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000
NOW-01462,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000283,0.000000,0.0,0.00000
SDN-41283,0.000000,0.000000,0.065,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000
NWY-41010,0.000000,0.006250,0.000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CEN-21245,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.150000,0.000000,0.000000,0.000000,0.0,0.00000
PAR-77883,0.416667,0.000125,0.000,0.0,0.0,0.0,0.000278,0.011667,0.000006,0.0,...,0.0,0.0,0.020625,0.0,0.016750,0.066667,0.000029,0.010667,0.0,0.00500
KIM-02098,20.833333,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000


### SSE 계산

In [212]:
sr_diff_ratio= ((sr_user_RDI- sr_user_intake)/ sr_user_RDI)/ 100.
sr_diff_ratio.fillna(0, inplace= True)
sr_diff_ratio

Vitamin_B12              -0.005833
Calcium                   0.006535
Iron                      0.000600
Leucine                  -0.004041
Histidine                -0.005992
Phenylalanine+Tyrosine   -0.000262
Magnesium                 0.005500
Selenium                 -0.002100
Copper                    0.009988
Alpha_Linolenic_Acid      0.003750
Threonine                -0.006573
Linoleic_Acid             0.001231
Methionine                0.004067
Phosphorus               -0.000169
Niacin                    0.000000
Zinc                     -0.008800
Sodium                   -0.003909
Vitamin_D                 0.000000
Vitamin_C                 0.008180
Lysine                   -0.002510
Valine                   -0.006525
Vitamin_A                -0.077735
Dietary_Fiber             0.004000
Vitamin_E                 0.000000
Vitamin_B6                0.000000
Potassium                 0.005377
Vitamin_K                 0.000000
Iso_Leucine              -0.006574
Manganese           

In [380]:
sr_diff_ratio[sr_diff_ratio> 0].sort_values()

Iron                    0.000600
Linoleic_Acid           0.001231
Manganese               0.002000
Alpha_Linolenic_Acid    0.003750
Dietary_Fiber           0.004000
Methionine              0.004067
Potassium               0.005377
Magnesium               0.005500
Calcium                 0.006535
Vitamin_C               0.008180
Copper                  0.009988
dtype: float64

In [211]:
def calculate_sse(x):
    SSE = np.sum((x - sr_diff_ratio)**2)
    return SSE

In [361]:
SSE_nutrition_supplements = df_ratio.apply(lambda row : calculate_sse(row), axis=1)

In [362]:
SSE_nutrition_supplements

prod_cd
NAP-03772      4.407078
THR-22102      0.008111
NOW-01462      0.006680
SDN-41283      0.010830
NWY-41010      0.006640
                ...    
CEN-21245      0.029183
PAR-77883      0.194804
KIM-02098    434.277516
NAU-62808      0.009958
SDN-90526      0.008365
Length: 2173, dtype: float64

In [363]:
SSE_nutrition_supplements.sort_values(ascending= True)

prod_cd
CAT-02554    6.566171e-03
AOR-08333    6.596103e-03
SOR-04420    6.603918e-03
SOR-04421    6.603918e-03
RLT-10102    6.606548e-03
                 ...     
SPS-90810    1.736604e+03
NWY-15685    2.500096e+03
NOR-01619    1.562501e+04
ARG-56380    1.777778e+08
SDN-50356    6.250000e+08
Length: 2173, dtype: float64

In [364]:
df_ratio.loc['CAT-02554']

Vitamin_B12               0.000000
Calcium                   0.009000
Iron                      0.000000
Leucine                   0.000000
Histidine                 0.000000
Phenylalanine+Tyrosine    0.000000
Magnesium                 0.001472
Selenium                  0.000000
Copper                    0.000000
Alpha_Linolenic_Acid      0.000000
Threonine                 0.000000
Linoleic_Acid             0.000000
Methionine                0.000000
Phosphorus                0.000000
Niacin                    0.000000
Zinc                      0.000000
Sodium                    0.000067
Vitamin_D                 0.000000
Vitamin_C                 0.009000
Lysine                    0.000000
Valine                    0.000000
Vitamin_A                 0.000000
Dietary_Fiber             0.000000
Vitamin_E                 0.000000
Vitamin_B6                0.000000
Potassium                 0.000000
Vitamin_K                 0.000000
Iso_Leucine               0.000000
Manganese           

In [366]:
df_ratio.loc['AOR-08333']

Vitamin_B12               0.000000
Calcium                   0.000000
Iron                      0.000000
Leucine                   0.000000
Histidine                 0.000000
Phenylalanine+Tyrosine    0.000000
Magnesium                 0.002778
Selenium                  0.000000
Copper                    0.000000
Alpha_Linolenic_Acid      0.000000
Threonine                 0.000000
Linoleic_Acid             0.000000
Methionine                0.000000
Phosphorus                0.000000
Niacin                    0.000000
Zinc                      0.000000
Sodium                    0.000000
Vitamin_D                 0.000000
Vitamin_C                 0.010000
Lysine                    0.000000
Valine                    0.000000
Vitamin_A                 0.000000
Dietary_Fiber             0.000000
Vitamin_E                 0.000000
Vitamin_B6                0.000000
Potassium                 0.000000
Vitamin_K                 0.000000
Iso_Leucine               0.000000
Manganese           

In [367]:
df.loc['CAT-02554'][df.loc['CAT-02554']>0], df.loc['AOR-08333'][df.loc['AOR-08333']>0]

(Calcium      720.0
 Magnesium     53.0
 Sodium        10.0
 Vitamin_C     90.0
 Name: CAT-02554, dtype: float64,
 Magnesium    100.0
 Vitamin_C    100.0
 Name: AOR-08333, dtype: float64)

### 코사인 유사도

In [316]:
from sklearn.metrics.pairwise import cosine_similarity

In [369]:
df_cosine_similarity = pd.concat([df_ratio, df_diff_ratio], join='outer')

In [370]:
df_cosine_similarity

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
NAP-03772,2.083333,0.000313,0.0850,0.000000,0.000000,0.000000,0.000000,0.001667,0.000024,0.00000,...,0.00000,0.000000,0.000000,0.000,0.016667,0.166667,0.000000,0.000000,0.000000,0.01250
THR-22102,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
NOW-01462,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000283,0.000000,0.000000,0.00000
SDN-41283,0.000000,0.000000,0.0650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
NWY-41010,0.000000,0.006250,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PAR-77883,0.416667,0.000125,0.0000,0.000000,0.000000,0.000000,0.000278,0.011667,0.000006,0.00000,...,0.00000,0.000000,0.020625,0.000,0.016750,0.066667,0.000029,0.010667,0.000000,0.00500
KIM-02098,20.833333,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
NAU-62808,0.025000,0.002375,0.0000,0.000000,0.000000,0.000000,0.008333,0.009167,0.000011,0.00000,...,0.00000,0.000000,0.000000,0.000,0.018333,0.026667,0.000214,0.016000,0.000000,0.00575
SDN-90526,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [323]:
df_cosine_similarity.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2174 entries, NAP-03772 to 0
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Vitamin_B12             2174 non-null   float64
 1   Calcium                 2174 non-null   float64
 2   Iron                    2174 non-null   float64
 3   Leucine                 2174 non-null   float64
 4   Histidine               2174 non-null   float64
 5   Phenylalanine+Tyrosine  2174 non-null   float64
 6   Magnesium               2174 non-null   float64
 7   Selenium                2174 non-null   float64
 8   Copper                  2174 non-null   float64
 9   Alpha_Linolenic_Acid    2174 non-null   float64
 10  Threonine               2174 non-null   float64
 11  Linoleic_Acid           2174 non-null   float64
 12  Methionine              2174 non-null   float64
 13  Phosphorus              2174 non-null   float64
 14  Niacin                  2174 non-null   

In [371]:
cosine_similarity= cosine_similarity(df_cosine_similarity, df_cosine_similarity)
cosine_similarity

array([[ 1.        ,  0.00239001,  0.        , ...,  0.48649391,
         0.01434005, -0.06944656],
       [ 0.00239001,  1.        ,  0.        , ...,  0.19464812,
         0.        , -0.10764945],
       [ 0.        ,  0.        ,  1.        , ...,  0.00379185,
         0.        ,  0.06577102],
       ...,
       [ 0.48649391,  0.19464812,  0.00379185, ...,  1.        ,
         0.31851511, -0.00883879],
       [ 0.01434005,  0.        ,  0.        , ...,  0.31851511,
         1.        ,  0.10006506],
       [-0.06944656, -0.10764945,  0.06577102, ..., -0.00883879,
         0.10006506,  1.        ]])

In [372]:
len(cosine_similarity[:,-1])

2174

In [373]:
df_cosine_similarity['cosine_similarity']= cosine_similarity[:,-1]

In [374]:
df_cosine_similarity.sort_values(by= 'cosine_similarity', ascending= False)

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese,cosine_similarity
0,-0.005833,0.006535,0.0006,-0.004041,-0.005992,-0.000262,0.005500,-0.0021,0.009988,0.00375,...,-0.006525,-0.077735,0.004,0.0,0.0,0.005377,0.0,-0.006574,0.002,1.000000
CAT-02554,0.000000,0.009000,0.0000,0.000000,0.000000,0.000000,0.001472,0.0000,0.000000,0.00000,...,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000,0.133921
PTG-04354,0.000000,0.002875,0.0000,0.000000,0.000000,0.000000,0.009722,0.0000,0.000000,0.00000,...,0.000000,0.000000,0.000,0.0,0.0,0.000314,0.0,0.000000,0.000,0.123637
PTG-00098,0.000000,0.002875,0.0000,0.000000,0.000000,0.000000,0.009722,0.0000,0.000000,0.00000,...,0.000000,0.000000,0.000,0.0,0.0,0.000314,0.0,0.000000,0.000,0.123637
NTB-72645,0.000000,0.000313,0.0000,0.000000,0.000000,0.000000,0.000278,0.0000,0.000000,0.00000,...,0.000000,0.000000,0.000,0.0,0.0,0.000283,0.0,0.000000,0.000,0.123365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NOW-00342,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,...,0.000000,0.093750,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000,-0.950924
NWY-40110,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,...,0.000000,0.037500,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000,-0.950924
NFS-01041,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,...,0.000000,0.012375,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000,-0.950924
CLF-05531,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,...,0.000000,0.037500,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000,-0.950924


In [353]:
df_diff_ratio.T

,0
Vitamin_B12,-0.005833
Calcium,0.006535
Iron,0.000600
Leucine,-0.004041
Histidine,-0.005992
Phenylalanine+Tyrosine,-0.000262
Magnesium,0.005500
Selenium,-0.002100
Copper,0.009988
Alpha_Linolenic_Acid,0.003750


In [339]:
df_ratio.loc['CAT-02554'][df_ratio.loc['CAT-02554']>0]

Calcium      0.009000
Magnesium    0.001472
Sodium       0.000067
Vitamin_C    0.009000
Name: CAT-02554, dtype: float64

In [338]:
df.loc['CAT-02554'][df.loc['CAT-02554']>0]

Calcium      720.0
Magnesium     53.0
Sodium        10.0
Vitamin_C     90.0
Name: CAT-02554, dtype: float64

In [340]:
df_ratio.loc['PTG-04354'][df_ratio.loc['PTG-04354']>0]

Calcium      0.002875
Magnesium    0.009722
Vitamin_D    0.002000
Vitamin_C    0.029000
Potassium    0.000314
Name: PTG-04354, dtype: float64

In [341]:
df.loc['PTG-04354'][df.loc['PTG-04354']>0]

Calcium      230.0
Magnesium    350.0
Vitamin_D      2.0
Vitamin_C    290.0
Potassium    110.0
Name: PTG-04354, dtype: float64

In [342]:
df_ratio.loc['PTG-00098'][df_ratio.loc['PTG-00098']>0]

Calcium      0.002875
Magnesium    0.009722
Vitamin_D    0.002000
Vitamin_C    0.029000
Potassium    0.000314
Name: PTG-00098, dtype: float64

In [343]:
df.loc['PTG-00098'][df.loc['PTG-00098']>0]

Calcium      230.0
Magnesium    350.0
Vitamin_D      2.0
Vitamin_C    290.0
Potassium    110.0
Name: PTG-00098, dtype: float64

In [375]:
sr_diff_ratio[sr_diff_ratio> 0]

Calcium                 0.006535
Iron                    0.000600
Magnesium               0.005500
Copper                  0.009988
Alpha_Linolenic_Acid    0.003750
Linoleic_Acid           0.001231
Methionine              0.004067
Vitamin_C               0.008180
Dietary_Fiber           0.004000
Potassium               0.005377
Manganese               0.002000
dtype: float64

In [330]:
#Counter(test_user_RDI)- Counter(test_user_intake)
(sr_user_RDI- sr_user_intake)[sr_user_RDI- sr_user_intake> 0]

Calcium                  522.8
Iron                       0.6
Magnesium                198.0
Copper                   849.0
Alpha_Linolenic_Acid       0.6
Linoleic_Acid              1.6
Methionine               569.4
Vitamin_C                 81.8
Dietary_Fiber             12.0
Potassium               1881.8
Manganese                  0.8
dtype: float64

In [356]:
df.sort_values(by= 'Potassium', ascending= False)

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
prod_cd,,,,,,,,,,,,,,,,,,,,,
PHI-01010,0.0,450.0,8.0,0.0,0.0,0.0,253.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,1260.0,0.0,0.0,0.900
LEM-10315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,1020.0,0.0,0.0,0.000
ZIP-00009,2500.0,60.0,0.0,0.0,0.0,0.0,105.0,27.5,0.25,0.0,...,0.0,0.0,0.00,0.0,7.5,0.85,950.0,0.0,0.0,1.150
ZIP-00037,2500.0,60.0,0.0,0.0,0.0,0.0,105.0,27.5,0.25,0.0,...,0.0,0.0,0.45,0.0,7.5,0.85,950.0,0.0,0.0,0.001
ZIP-00024,2500.0,60.0,0.0,0.0,0.0,0.0,105.0,27.5,0.25,0.0,...,0.0,0.0,0.00,0.0,7.5,0.85,950.0,0.0,0.0,1.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MCL-01732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000
MAV-05171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000
NAP-29927,6.0,50.0,5.0,0.0,0.0,0.0,10.0,35.0,1.00,0.0,...,0.0,0.0,0.00,0.0,20.0,2.00,0.0,40.0,0.0,1.000


In [357]:
df_ratio.loc['PHI-01010'][df_ratio.loc['PHI-01010']>0]

Calcium       0.005625
Iron          0.008000
Magnesium     0.007028
Phosphorus    0.003343
Vitamin_D     0.009000
Potassium     0.003600
Manganese     0.002250
Name: PHI-01010, dtype: float64

In [358]:
df.loc['PHI-01010'][df.loc['PHI-01010']>0]

Calcium        450.0
Iron             8.0
Magnesium      253.0
Phosphorus     234.0
Vitamin_D        9.0
Potassium     1260.0
Manganese        0.9
Name: PHI-01010, dtype: float64

### CSV 

In [201]:
# 'amino_acids', 'minerals', 'vitamins'
csv= preprocess_nutrition_supplements(json_path, ['amino-acids'])

In [202]:
df_csv= pd.DataFrame(csv, dtype= float, columns= ['prod_cd']+total)

/var/folders/pr/phpb7l1s675c1wqrhj9jc4lh0000gn/T/ipykernel_20943/3351190882.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df_csv= pd.DataFrame(csv, dtype= float, columns= ['prod_cd']+total)


In [203]:
df_csv.columns

Index(['prod_cd', 'Protein', 'Fat', 'Carbohydrate', 'Dietary_Fiber', 'Calcium',
       'Iron', 'Magnesium', 'Phosphorus', 'Potassium', 'Sodium', 'Zinc',
       'Copper', 'Manganese', 'Selenium', 'Vitamin_A', 'Vitamin_D', 'Niacin',
       'Folic_acid', 'Vitamin_B12', 'Vitamin_B6', 'Vitamin_C', 'Vitamin_E',
       'Vitamin_K', 'Leucine', 'Iso_Leucine', 'Histidine', 'Linoleic_Acid',
       'Alpha_Linolenic_Acid', 'Lysine', 'Methionine', 'Phenylalanine',
       'Tyrosine', 'Threonine', 'Valine', 'Cholesterol', 'Calories'],
      dtype='object')

In [204]:
df_csv.set_index('prod_cd', inplace= True)
df_csv.fillna(0, inplace= True)
df_csv['Phenylalanine+Tyrosine']= df_csv['Phenylalanine']+ df_csv['Tyrosine']
df_csv.drop(['Tyrosine'], axis=1, inplace= True)
df_csv.drop(['Phenylalanine'], axis=1, inplace= True)
df_csv

,Protein,Fat,Carbohydrate,Dietary_Fiber,Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,...,Histidine,Linoleic_Acid,Alpha_Linolenic_Acid,Lysine,Methionine,Threonine,Valine,Cholesterol,Calories,Phenylalanine+Tyrosine
prod_cd,,,,,,,,,,,,,,,,,,,,,
EVL-90914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1250.0,0.0,0.0,0.00
JYM-02141,0.0,0.0,1.0,0.0,41.0,0.0,0.0,0.0,0.0,80.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1200.0,0.0,0.0,0.00
NOW-00265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.35
EVL-02003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1250.0,0.0,0.0,0.00
LEX-12539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,600.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LEX-16781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,620.0,0.0,0.0,0.0,0.0,0.0,0.00
MSF-00382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,0.00
THR-51403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50


In [205]:
df_csv.to_csv('./amino-acids.csv')

In [289]:
#dfa2= pd.DataFrame(dfa.sort_values(by= 'price', ascending=False)[:10], columns= standard_columns, dtype= float)
dfa2= dfa.sort_values(by= 'price', ascending=False)[:20]
dfa2= dfa2[standard_columns]

In [290]:
dfv2= dfv.sort_values(by= 'price', ascending=False)[:20]
dfv2= dfv2[standard_columns]

In [291]:
dfm2= dfm.sort_values(by= 'price', ascending=False)[:20]
dfm2= dfm2[standard_columns]

In [292]:
#dfm2.index[0]+
#dfa2.index[0]+
dfv2.index[0]

'MCL-01996'

In [294]:
test= dict()
for i in range(20):
    for j in range(20):
        for k in range(20):
            test[f'{dfm2.index[i]}+{dfv2.index[j]}+{dfa2.index[k]}']= dfm2.iloc[i,:]+dfv2.iloc[j,:]+dfa2.iloc[k,:]

In [295]:
len(test)

8000

In [296]:
test

{'MCL-03069+MCL-01996+OLY-00399': Vitamin_B12                  0.0
 Calcium                      0.0
 Iron                         0.0
 Leucine                      0.0
 Histidine                    0.0
 Phenylalanine+Tyrosine       0.0
 Magnesium                 2000.0
 Selenium                     0.0
 Copper                       0.0
 Alpha_Linolenic_Acid         0.0
 Threonine                    0.0
 Linoleic_Acid                0.0
 Methionine                   0.0
 Phosphorus                   0.0
 Niacin                       0.0
 Zinc                        18.0
 Sodium                       0.0
 Vitamin_D                  125.0
 Vitamin_C                   55.0
 Lysine                     730.0
 Valine                       0.0
 Vitamin_A                    0.0
 Dietary_Fiber                0.0
 Vitamin_E                    0.0
 Vitamin_B6                   0.0
 Potassium                    0.0
 Vitamin_K                  180.0
 Iso_Leucine                  0.0
 Manganese     

In [297]:
df_test= pd.DataFrame(test)

In [298]:
df_test.T

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
MCL-03069+MCL-01996+OLY-00399,0.0,0.0,0.0,0.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,730.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,0.0,0.0
MCL-03069+MCL-01996+MRM-71027,0.0,0.0,0.0,2500.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,0.0,1500.0,0.0,0.0,0.0,2.0,0.0,180.0,1000.0,0.0
MCL-03069+MCL-01996+OPN-02522,0.0,0.0,0.0,2500.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,0.0,0.0,0.0,180.0,1250.0,0.0
MCL-03069+MCL-01996+EVL-86451,0.0,180.0,0.0,2500.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,40.0,0.0,150.0,180.0,1250.0,0.0
MCL-03069+MCL-01996+THR-00464,0.0,0.0,0.0,1250.0,150.0,0.13,2000.0,0.0,0.0,0.0,...,650.0,625.0,0.0,0.0,0.0,0.0,0.0,180.0,625.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JIG-00153+NOR-01806+MBM-00148,0.0,0.0,0.0,0.0,0.0,0.50,500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
JIG-00153+NOR-01806+UNN-02701,0.0,0.0,0.0,620.0,110.0,0.36,500.0,0.0,0.0,0.0,...,520.0,330.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,0.0
JIG-00153+NOR-01806+MBM-00140,0.0,0.0,0.0,0.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JIG-00153+NOR-01806+EVL-86422,30.0,0.0,0.0,2500.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,0.0,10.0,0.0,0.0,1250.0,0.0


In [299]:
SSE_test = df_test.T.apply(lambda row : calculate_sse(row), axis=1)

In [300]:
SSE_test.sort_values(ascending= True)
#SSE_test

MCL-03256+NOR-01806+MBM-00148    7.124377e+03
DWM-24150+NOR-01806+MBM-00148    8.848614e+03
MCL-03256+CAR-17520+MBM-00148    8.999377e+03
MCL-03256+INN-40012+MBM-00148    9.299960e+03
DWM-24150+CAR-17520+MBM-00148    1.072361e+04
                                     ...     
MCL-01995+THR-17001+DYZ-18102    2.901693e+08
MCL-01995+THR-17001+DYZ-18101    2.901693e+08
GOL-11761+THR-17001+DYZ-18102    2.979895e+08
GOL-11761+THR-17001+DYZ-18101    2.979895e+08
GOL-11761+THR-17001+DYZ-18100    2.979895e+08
Length: 8000, dtype: float64

In [309]:
df_test['DWM-24150+NOR-01806+MBM-00148'][df_test['DWM-24150+NOR-01806+MBM-00148']> 0].sort_values(ascending=False)

Magnesium                 75.0
Vitamin_C                 50.0
Vitamin_D                 25.0
Vitamin_B6                10.0
Phenylalanine+Tyrosine     0.5
Name: DWM-24150+NOR-01806+MBM-00148, dtype: float64

In [310]:
df_test['MCL-03256+CAR-17520+MBM-00148'][df_test['MCL-03256+CAR-17520+MBM-00148']> 0].sort_values(ascending=False)

Magnesium                 80.0
Vitamin_D                 50.0
Vitamin_B6                10.0
Phenylalanine+Tyrosine     0.5
Name: MCL-03256+CAR-17520+MBM-00148, dtype: float64

In [312]:
df_test.T

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
MCL-03069+MCL-01996+OLY-00399,0.0,0.0,0.0,0.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,730.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,0.0,0.0
MCL-03069+MCL-01996+MRM-71027,0.0,0.0,0.0,2500.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,0.0,1500.0,0.0,0.0,0.0,2.0,0.0,180.0,1000.0,0.0
MCL-03069+MCL-01996+OPN-02522,0.0,0.0,0.0,2500.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,0.0,0.0,0.0,180.0,1250.0,0.0
MCL-03069+MCL-01996+EVL-86451,0.0,180.0,0.0,2500.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,40.0,0.0,150.0,180.0,1250.0,0.0
MCL-03069+MCL-01996+THR-00464,0.0,0.0,0.0,1250.0,150.0,0.13,2000.0,0.0,0.0,0.0,...,650.0,625.0,0.0,0.0,0.0,0.0,0.0,180.0,625.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JIG-00153+NOR-01806+MBM-00148,0.0,0.0,0.0,0.0,0.0,0.50,500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
JIG-00153+NOR-01806+UNN-02701,0.0,0.0,0.0,620.0,110.0,0.36,500.0,0.0,0.0,0.0,...,520.0,330.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,0.0
JIG-00153+NOR-01806+MBM-00140,0.0,0.0,0.0,0.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JIG-00153+NOR-01806+EVL-86422,30.0,0.0,0.0,2500.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,0.0,10.0,0.0,0.0,1250.0,0.0


In [313]:
# df_test.fillna(0, inplace=True)
# df_diff.fillna(0, inplace=True)
df_diff.index= ['test']

df_cosine_similarity = pd.concat([df_test.T, df_diff], join='outer')


In [314]:
df_cosine_similarity

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Lysine,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese
MCL-03069+MCL-01996+OLY-00399,0.0,0.0,0.0,0.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,730.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,0.0,0.0
MCL-03069+MCL-01996+MRM-71027,0.0,0.0,0.0,2500.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,0.0,1500.0,0.0,0.0,0.0,2.0,0.0,180.0,1000.0,0.0
MCL-03069+MCL-01996+OPN-02522,0.0,0.0,0.0,2500.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,0.0,0.0,0.0,180.0,1250.0,0.0
MCL-03069+MCL-01996+EVL-86451,0.0,180.0,0.0,2500.0,0.0,0.00,2000.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,40.0,0.0,150.0,180.0,1250.0,0.0
MCL-03069+MCL-01996+THR-00464,0.0,0.0,0.0,1250.0,150.0,0.13,2000.0,0.0,0.0,0.0,...,650.0,625.0,0.0,0.0,0.0,0.0,0.0,180.0,625.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JIG-00153+NOR-01806+UNN-02701,0.0,0.0,0.0,620.0,110.0,0.36,500.0,0.0,0.0,0.0,...,520.0,330.0,0.0,0.0,0.0,0.0,0.0,0.0,350.0,0.0
JIG-00153+NOR-01806+MBM-00140,0.0,0.0,0.0,0.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JIG-00153+NOR-01806+EVL-86422,30.0,0.0,0.0,2500.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,0.0,10.0,0.0,0.0,1250.0,0.0
JIG-00153+NOR-01806+EVL-86424,30.0,0.0,0.0,2500.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,1250.0,0.0,0.0,0.0,10.0,0.0,0.0,1250.0,0.0


In [317]:
cs_test= cosine_similarity(df_cosine_similarity, df_cosine_similarity)
cs_test

array([[ 1.        ,  0.51365302,  0.51603921, ...,  0.15168335,
         0.15163299, -0.01113384],
       [ 0.51365302,  1.        ,  0.99537594, ...,  0.90836839,
         0.90806684, -0.20538975],
       [ 0.51603921,  0.99537594,  1.        , ...,  0.91258649,
         0.91228354, -0.20451307],
       ...,
       [ 0.15168335,  0.90836839,  0.91258649, ...,  1.        ,
         0.99966803, -0.25465667],
       [ 0.15163299,  0.90806684,  0.91228354, ...,  0.99966803,
         1.        , -0.25671922],
       [-0.01113384, -0.20538975, -0.20451307, ..., -0.25465667,
        -0.25671922,  1.        ]])

In [318]:
df_cosine_similarity['cosine_similarity']= cs_test[:,-1]

In [319]:
df_cosine_similarity.sort_values(by= 'cosine_similarity', ascending= False)

,Vitamin_B12,Calcium,Iron,Leucine,Histidine,Phenylalanine+Tyrosine,Magnesium,Selenium,Copper,Alpha_Linolenic_Acid,...,Valine,Vitamin_A,Dietary_Fiber,Vitamin_E,Vitamin_B6,Potassium,Vitamin_K,Iso_Leucine,Manganese,cosine_similarity
test,-1.4,522.8,0.6,-1252.6,-599.2,-94.20,198.0,-12.6,849.0,0.6,...,-1109.2,-6218.8,12.0,0.0,0.0,1881.8,0.0,-920.4,0.8,1.000000
MBM-00426+NOR-01806+MBM-00148,0.0,0.0,0.0,0.0,0.0,0.50,125.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,99.0,0.0,0.0,0.0,0.185789
MBM-00426+CAR-17520+MBM-00148,0.0,0.0,0.0,0.0,0.0,0.50,125.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,99.0,0.0,0.0,0.0,0.179489
MBM-00426+INN-40012+MBM-00148,50.0,0.0,0.0,0.0,0.0,0.50,125.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.0,99.0,0.0,0.0,0.0,0.178481
MBM-00426+NOR-01806+MBM-00140,0.0,0.0,0.0,0.0,0.0,0.00,125.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,0.134287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DWM-24150+SOL-02072+THR-00366,100.0,35.0,9.0,1250.0,150.0,0.13,112.0,0.0,1.5,0.0,...,625.0,4500.0,0.0,201.0,50.0,10.0,20.0,625.0,2.0,-0.930676
CAT-02683+SOL-02072+THR-00366,100.0,35.0,9.0,1250.0,150.0,0.13,147.0,0.0,1.5,0.0,...,625.0,4500.0,0.0,201.0,50.0,10.0,20.0,625.0,2.0,-0.931135
CAT-02683+SOL-02072+THR-00464,100.0,35.0,9.0,1250.0,150.0,0.13,147.0,0.0,1.5,0.0,...,625.0,4500.0,0.0,201.0,50.0,10.0,20.0,625.0,2.0,-0.931135
MCL-03256+SOL-02072+THR-00366,100.0,35.0,9.0,1250.0,150.0,0.13,117.0,0.0,1.5,0.0,...,625.0,4500.0,0.0,201.0,50.0,10.0,20.0,625.0,2.0,-0.931393
